# This Notebook prepares the MySQL Databases for the MOVES runs 

## User-defined parameters

In [1]:
pollutants = []
years = []
counties = []

#Need to add a variable for if the model run is for conformity or for SIP

year = 2050
countyid = 49049
monthid = 1
dayid = 5


## Load Libraries

In [2]:
import pandas as pd
import pathlib
import os
from IPython.display import display, HTML
import numpy as np
import mariadb
import sys
import mysql.connector

## Input files

In [3]:
#Referece file directory path
file_dir_path = pathlib.Path(os.getcwd(),"Input/AgePopCheckerFiles")

#Reference File Names
default_agedistribution_name = 'sourcetypeagedistribution_4.csv'
dmv_agedistribution_name = 'sourcetypeagedistribution_conformity_4.csv'
sourcetypeyear_name = 'sourcetypepopulation_2022_4.csv'

vmtbyveh_dist_county_name = '2mv_vmt_byveh_dist_county.csv'

county_filename = 'county.csv'
sourcetypeage_movesdb_filename = 'sourcetypeage_movesdb.csv'
basefiles_pop_vmt_filename = 'basefiles_pop_vmt.csv'

pd.reset_option('display')

# Age Distribution Fixer

### This part is supposed to replace the age distributions for all vehicles with the dmv age distributions, except for vehicles 53 and 62, which should use the default age distributions

In [4]:
# Read in age distribution file
default_agedistribution_filepath = pathlib.Path(file_dir_path,default_agedistribution_name)
dmv_agedistribution_filepath = pathlib.Path(file_dir_path,dmv_agedistribution_name)


default_agedistribution_df = pd.read_csv(default_agedistribution_filepath)
dmv_agedistribution_df = pd.read_csv(dmv_agedistribution_filepath)

display(default_agedistribution_df)
display(dmv_agedistribution_df)


,sourceTypeID,yearID,ageID,ageFraction
0,11,1990,0,0.051387
1,11,1990,1,0.050995
2,11,1990,2,0.063651
3,11,1990,3,0.079356
4,11,1990,4,0.073790
...,...,...,...,...
25384,62,2060,26,0.018751
25385,62,2060,27,0.017305
25386,62,2060,28,0.015933
25387,62,2060,29,0.014621


,countyID,sourceTypeID,yearID,ageID,ageFraction
0,49003,11,2023,0,0.054938
1,49003,11,2023,1,0.063955
2,49003,11,2023,2,0.033286
3,49003,11,2023,3,0.026357
4,49003,11,2023,4,0.027298
...,...,...,...,...,...
99995,49057,43,2056,20,0.025150
99996,49057,43,2056,21,0.023793
99997,49057,43,2056,22,0.022333
99998,49057,43,2056,23,0.020963


In [5]:
# Filter dmv data to only records in selected year and county to not type 53 and 62
dmv_agedistribution_filtered = dmv_agedistribution_df.loc[(dmv_agedistribution_df['yearID'] == year) & (dmv_agedistribution_df['sourceTypeID'] != 53) & (dmv_agedistribution_df['sourceTypeID'] != 62) & (dmv_agedistribution_df['countyID'] == countyid)]


display(dmv_agedistribution_filtered)

,countyID,sourceTypeID,yearID,ageID,ageFraction
77407,49049,11,2050,0,0.055289
77408,49049,11,2050,1,0.055199
77409,49049,11,2050,2,0.054159
77410,49049,11,2050,3,0.051383
77411,49049,11,2050,4,0.048673
...,...,...,...,...,...
90391,49049,61,2050,26,0.017993
90392,49049,61,2050,27,0.016627
90393,49049,61,2050,28,0.025721
90394,49049,61,2050,29,0.015932


In [6]:
# Filter default data to records to vehicle types 53 and 62 from selected year
#default_agedistribution_filtered = default_agedistribution_df.loc[(default_agedistribution_df['yearID'] == year) & (default_agedistribution_df['sourceTypeID'] == 53) & (default_agedistribution_df['sourceTypeID'] == 62)]

default_agedistribution_filtered = default_agedistribution_df.loc[(default_agedistribution_df['yearID'] == year) & ((default_agedistribution_df['sourceTypeID'] == 53) | (default_agedistribution_df['sourceTypeID'] == 62))]


#display(default_agedistribution_df.dtypes)
display(default_agedistribution_filtered)


,sourceTypeID,yearID,ageID,ageFraction
19189,53,2050,0,0.043878
19190,53,2050,1,0.043380
19191,53,2050,2,0.043008
19192,53,2050,3,0.042592
19193,53,2050,4,0.042256
...,...,...,...,...
25074,62,2050,26,0.017959
25075,62,2050,27,0.016597
25076,62,2050,28,0.015040
25077,62,2050,29,0.024197


In [7]:
#Join the two dataframes
joined_agedistribution_df = pd.concat([dmv_agedistribution_filtered,default_agedistribution_filtered])


joined_agedistribution_df.drop(columns=['countyID'], inplace=True)
joined_agedistribution_df.sort_values(by = 'sourceTypeID', inplace=True)

display(joined_agedistribution_df)


,sourceTypeID,yearID,ageID,ageFraction
77407,11,2050,0,0.055289
77437,11,2050,30,0.056000
77436,11,2050,29,0.012550
77435,11,2050,28,0.022450
77434,11,2050,27,0.014461
...,...,...,...,...
25050,62,2050,2,0.039959
25049,62,2050,1,0.039080
25077,62,2050,29,0.024197
25062,62,2050,14,0.034981


# Population Checker

### This script will create a new population for 53 and 62, based on the following inputs: TDM VMT and age distributions for all source types, Moves4 default VMT and populations.  If new population is < the existing population the database will not be deleted and the original population will be in the database

In [8]:
# Read in age distribution file
sourcetypeyear_filepath = pathlib.Path(file_dir_path,sourcetypeyear_name)

sourcetypeyear_df = pd.read_csv(sourcetypeyear_filepath)

display(sourcetypeyear_df)


,yearID,countyid,sourceTypeID,pop
0,2014,49001,11,109.000
1,2015,49001,11,104.000
2,2016,49001,11,102.000
3,2017,49001,11,124.000
4,2018,49001,11,136.000
...,...,...,...,...
17714,2056,49057,62,974.196
17715,2057,49057,62,978.659
17716,2058,49057,62,983.141
17717,2059,49057,62,987.644


In [9]:
# Filter to only records in 2050 not type 53 and 62
filtered_sourcetype = sourcetypeyear_df.loc[(sourcetypeyear_df['yearID'] == year) & (sourcetypeyear_df['countyid'] == countyid)]

display(filtered_sourcetype)

,yearID,countyid,sourceTypeID,pop
14700,2050,49049,11,17227.400
14747,2050,49049,21,103123.000
14794,2050,49049,31,388007.000
14841,2050,49049,32,44533.100
14888,2050,49049,41,750.023
14935,2050,49049,42,105.885
14982,2050,49049,43,927.970
15029,2050,49049,51,167.327
15076,2050,49049,52,29472.900
15123,2050,49049,53,4748.740


## VMT Calc 1

In [10]:
#VMT Calc 1 sums the sourcetype df by groups: 11, 21-32, 41-43, 51-54, and 61-62
group_dict = {
    11: [11],
    21: [21,31,32],
    31: [21,31,32],
    32: [21,31,32],
    41: [41,42,43],
    42: [41,42,43],
    43: [41,42,43],
    51: [51,52,53,54],
    52: [51,52,53,54],
    53: [51,52,53,54],
    54: [51,52,53,54],
    61: [61,62],
    62: [61,62]

}

vmt_calc_1 = filtered_sourcetype.copy()

for sourcetype in filtered_sourcetype['sourceTypeID']:
    filtered_df = filtered_sourcetype[filtered_sourcetype['sourceTypeID'].isin(group_dict[sourcetype])]
    group_pop = filtered_df['pop'].sum()
    vmt_calc_1.loc[vmt_calc_1['sourceTypeID'] == sourcetype,'pop'] = group_pop

display(vmt_calc_1)





,yearID,countyid,sourceTypeID,pop
14700,2050,49049,11,17227.400
14747,2050,49049,21,535663.100
14794,2050,49049,31,535663.100
14841,2050,49049,32,535663.100
14888,2050,49049,41,1783.878
14935,2050,49049,42,1783.878
14982,2050,49049,43,1783.878
15029,2050,49049,51,37620.057
15076,2050,49049,52,37620.057
15123,2050,49049,53,37620.057


## VMT Calc 2

In [11]:
#Create VMT_calc_02

#First, create htpms vtypeday
#Read in file 2 from 'File Preparation' notebook

df_2mv = pd.read_csv(vmtbyveh_dist_county_name)

#This resets the HPMS W and S values to just be in the same column, but with different monthIDs
hpmsvtypeday_win = df_2mv[['County_FIPS','yearID', 'monthID','dayID','HPMSVtypeID','HPMS_VMT_W']]
hpmsvtypeday_filtered_win = hpmsvtypeday_win.loc[hpmsvtypeday_win['County_FIPS']== countyid - 49000]
hpmsvtypeday_filtered_win = hpmsvtypeday_filtered_win.rename(columns={"HPMS_VMT_W":"VMT"})
hpmsvtypeday_filtered_win.loc[:,'monthID'] = 1
#display(hpmsvtypeday_filtered_win)

#Make df for summer VMT
hpmsvtypeday_sum = df_2mv[['County_FIPS','yearID', 'monthID','dayID','HPMSVtypeID','HPMS_VMT_S']]
hpmsvtypeday_filtered_sum = hpmsvtypeday_sum.loc[hpmsvtypeday_win['County_FIPS']== countyid - 49000]
hpmsvtypeday_filtered_sum = hpmsvtypeday_filtered_sum.rename(columns={"HPMS_VMT_S":"VMT"})
#display(hpmsvtypeday_filtered_sum)

#Join the two dfs into one
hpmsvtypeday = pd.concat([hpmsvtypeday_filtered_win,hpmsvtypeday_filtered_sum],axis=0)
display(hpmsvtypeday)


,County_FIPS,yearID,monthID,dayID,HPMSVtypeID,VMT
15,49,2050,1,5,10,40749.0
16,49,2050,1,5,25,16113052.0
17,49,2050,1,5,40,93697.0
18,49,2050,1,5,50,614852.0
19,49,2050,1,5,60,591856.0
15,49,2050,7,5,10,46003.0
16,49,2050,7,5,25,18190578.0
17,49,2050,7,5,40,106070.0
18,49,2050,7,5,50,696054.0
19,49,2050,7,5,60,670020.0


In [12]:
#Create VMT_Calc_02

#It comes from these tables:
#From
# rtp24_2050_pm25_m4_in.sourcetypeagedistribution CA,
# rtp24_2050_pm25_m4_in_pop_checker.sourcetypeyear CB,
# rtp24_2050_pm25_m4_in_pop_checker.vmt_calc_01 CC,
# rtp24_2050_pm25_m4_in.hpmsvtypeday CD,
# rtp24_2050_pm25_m4_in.county CF,
# movesdb20240104.sourcetypeage CE

#Read in "county" file

county_filepath = pathlib.Path(file_dir_path,county_filename)
county_df = pd.read_csv(county_filepath)

#Read in MovesDB sourcetypeage file

sourcetypeage_movesdb_filepath = pathlib.Path(file_dir_path,sourcetypeage_movesdb_filename)
sourcetypeage_movesdb_df = pd.read_csv(sourcetypeage_movesdb_filepath)

ca = joined_agedistribution_df
cb = filtered_sourcetype
cc = vmt_calc_1
cd = hpmsvtypeday
cf = county_df
ce = sourcetypeage_movesdb_df



In [13]:
# Display DFs
html_ca = ca.to_html(classes='scrollable',index = False)
html_cb = cb.to_html(classes='scrollable',index=False)
style = """
<style>
.scrollable {
    height: 400px
    overflow-y: scroll;
    display: block
}

table {
    width: 100%;
    border-collapse: collapse;
}
th, td {
    border: 1px solid #ddd;
    padding: 8px;
}
th {
    background-color: #f2f2f2;
}
</style>
"""
#display(HTML(style + html_ca))
#display(HTML(style + html_cb))
display(joined_agedistribution_df)

,sourceTypeID,yearID,ageID,ageFraction
77407,11,2050,0,0.055289
77437,11,2050,30,0.056000
77436,11,2050,29,0.012550
77435,11,2050,28,0.022450
77434,11,2050,27,0.014461
...,...,...,...,...
25050,62,2050,2,0.039959
25049,62,2050,1,0.039080
25077,62,2050,29,0.024197
25062,62,2050,14,0.034981


In [14]:
# Add Columns

# Create VMT_Calc_02 table
vmt_calc_02 = joined_agedistribution_df.copy()

# Add CountyID
vmt_calc_02['countyid'] = countyid

# Add MonthID
vmt_calc_02['monthid'] = monthid

# Add DayID
vmt_calc_02['dayid'] = dayid

#Join sourceTypePopulation
vmt_calc_02 = pd.merge(vmt_calc_02, cb[['sourceTypeID', 'pop']], on='sourceTypeID', how='left')

#Join vmtCalc1 as HPMSVtypePopulation
cc.rename(columns={'pop':'HPMSVtypePopulation'}, inplace=True)
vmt_calc_02 = pd.merge(vmt_calc_02, cc[['sourceTypeID', 'HPMSVtypePopulation']], on='sourceTypeID', how='left')

#Calculate SourceTypeAgePopulation
vmt_calc_02['sourcetypeagepopulation'] = vmt_calc_02['ageFraction'] * vmt_calc_02['pop']

#Join HPMSVtypeID and VMT
HPMSVTypeDict = {
    11: 10,
    21: 25,
    31: 25,
    32: 25,
    41: 40,
    42: 40,
    43: 40,
    51: 50,
    52: 50,
    53: 50,
    54: 50,
    61: 60,
    62: 60

}

vmt_calc_02.rename(columns={'monthid':'monthID', 'dayid':'dayID'}, inplace=True)
vmt_calc_02['HPMSVtypeID'] = vmt_calc_02['sourceTypeID'].map(HPMSVTypeDict)
HPMS_vmt = cd.loc[:,['HPMSVtypeID','VMT','monthID','dayID']]
vmt_calc_02 = pd.merge(left=vmt_calc_02, right=HPMS_vmt, how='left', left_on=['HPMSVtypeID','monthID','dayID'], right_on=['HPMSVtypeID', 'monthID','dayID'])

#Add relativeMAR
age_id = ce.loc[:,['ageID','sourceTypeID','relativeMAR']]
vmt_calc_02 = pd.merge(left=vmt_calc_02, right=age_id, how='left', left_on=['ageID','sourceTypeID'], right_on=['ageID','sourceTypeID'])

#Calculate HPMSvehFraction
vmt_calc_02['HPMSvehFraction'] = vmt_calc_02['ageFraction'] * vmt_calc_02['pop'] / vmt_calc_02['HPMSVtypePopulation']

#Calculate FractTemp
vmt_calc_02['FractTemp'] = vmt_calc_02['relativeMAR'] * vmt_calc_02['HPMSvehFraction']


scrollable_vmt_calc_02 = vmt_calc_02.to_html(classes='scrollable',index=False)
display(HTML(style + scrollable_vmt_calc_02))

sourceTypeID,yearID,ageID,ageFraction,countyid,monthID,dayID,pop,HPMSVtypePopulation,sourcetypeagepopulation,HPMSVtypeID,VMT,relativeMAR,HPMSvehFraction,FractTemp
11,2050,0,0.055289,49049,1,5,17227.400,17227.400,952.490887,10,40749.0,1.000000,0.055289,0.055289
11,2050,30,0.056000,49049,1,5,17227.400,17227.400,964.734400,10,40749.0,0.061000,0.056000,0.003416
11,2050,29,0.012550,49049,1,5,17227.400,17227.400,216.205593,10,40749.0,0.066000,0.012550,0.000828
11,2050,28,0.022450,49049,1,5,17227.400,17227.400,386.751685,10,40749.0,0.071000,0.022450,0.001594
11,2050,27,0.014461,49049,1,5,17227.400,17227.400,249.121986,10,40749.0,0.075000,0.014461,0.001085
11,2050,26,0.015860,49049,1,5,17227.400,17227.400,273.235178,10,40749.0,0.081000,0.015860,0.001285
11,2050,25,0.017054,49049,1,5,17227.400,17227.400,293.790911,10,40749.0,0.088000,0.017054,0.001501
11,2050,24,0.018180,49049,1,5,17227.400,17227.400,313.195855,10,40749.0,0.094000,0.018180,0.001709
11,2050,23,0.019178,49049,1,5,17227.400,17227.400,330.388800,10,40749.0,0.100000,0.019178,0.001918
11,2050,21,0.021295,49049,1,5,17227.400,17227.400,366.855760,10,40749.0,0.109000,0.021295,0.002321


## VMT Calc 3

In [15]:
vmt_calc_03 = vmt_calc_02.groupby('HPMSVtypeID')['FractTemp'].sum()
vmt_calc_03.rename('HPMSFractTemp', inplace=True)
display(vmt_calc_03)

HPMSVtypeID
10    0.245002
25    0.731527
40    0.452816
50    0.333292
60    0.353205
Name: HPMSFractTemp, dtype: float64

## VMT Calc 4

In [16]:
vmt_calc_04 = pd.merge(left=vmt_calc_02, right=vmt_calc_03, how='left', left_on=['HPMSVtypeID'], right_on=['HPMSVtypeID'])
vmt_calc_04['travelFraction'] = vmt_calc_04['FractTemp'] / vmt_calc_04 ['HPMSFractTemp']
vmt_calc_04['vmt'] = vmt_calc_04['VMT'] * (vmt_calc_04['FractTemp'] / vmt_calc_04['HPMSFractTemp'])
display(vmt_calc_04)


,sourceTypeID,yearID,ageID,ageFraction,countyid,monthID,dayID,pop,HPMSVtypePopulation,sourcetypeagepopulation,HPMSVtypeID,VMT,relativeMAR,HPMSvehFraction,FractTemp,HPMSFractTemp,travelFraction,vmt
0,11,2050,0,0.055289,49049,1,5,17227.40,17227.40,952.490887,10,40749.0,1.000000,0.055289,0.055289,0.245002,0.225669,9195.779938
1,11,2050,30,0.056000,49049,1,5,17227.40,17227.40,964.734400,10,40749.0,0.061000,0.056000,0.003416,0.245002,0.013943,568.153056
2,11,2050,29,0.012550,49049,1,5,17227.40,17227.40,216.205593,10,40749.0,0.066000,0.012550,0.000828,0.245002,0.003381,137.764906
3,11,2050,28,0.022450,49049,1,5,17227.40,17227.40,386.751685,10,40749.0,0.071000,0.022450,0.001594,0.245002,0.006506,265.105242
4,11,2050,27,0.014461,49049,1,5,17227.40,17227.40,249.121986,10,40749.0,0.075000,0.014461,0.001085,0.245002,0.004427,180.385273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,62,2050,2,0.039959,49049,1,5,1875.23,5006.31,74.931468,60,591856.0,1.000000,0.014967,0.014967,0.353205,0.042376,25080.500964
399,62,2050,1,0.039080,49049,1,5,1875.23,5006.31,73.283407,60,591856.0,1.000000,0.014638,0.014638,0.353205,0.041444,24528.874327
400,62,2050,29,0.024197,49049,1,5,1875.23,5006.31,45.374291,60,591856.0,0.070320,0.009063,0.000637,0.353205,0.001804,1067.981530
401,62,2050,14,0.034981,49049,1,5,1875.23,5006.31,65.597968,60,591856.0,0.419949,0.013103,0.005503,0.353205,0.015579,9220.602737


## VMT Calc 5

In [17]:
vmt_calc_05 = vmt_calc_04.loc[:,['sourceTypeID','countyid','yearID','monthID','dayID','vmt']]
vmt_calc_05 = vmt_calc_05.groupby(['countyid','yearID','monthID','dayID','sourceTypeID']).sum()
vmt_calc_05 = vmt_calc_05.reset_index()
display(vmt_calc_05)

,countyid,yearID,monthID,dayID,sourceTypeID,vmt
0,49049,2050,1,5,11,4.074900e+04
1,49049,2050,1,5,21,3.107127e+06
2,49049,2050,1,5,31,1.166752e+07
3,49049,2050,1,5,32,1.338401e+06
4,49049,2050,1,5,41,5.972358e+04
5,49049,2050,1,5,42,8.469174e+03
6,49049,2050,1,5,43,2.550425e+04
7,49049,2050,1,5,51,4.027610e+03
8,49049,2050,1,5,52,4.783957e+05
9,49049,2050,1,5,53,1.102842e+05


## VMT Calc 6

In [18]:
# Read in age distribution file
basefiles_pop_vmt_filepath = pathlib.Path(file_dir_path,basefiles_pop_vmt_filename)

mo = pd.read_csv(basefiles_pop_vmt_filepath, usecols=['yearID', 'countyID', 'sourceTypeID', 'activitytypeID','activity'])

vmt_calc_06 = mo.loc[(mo['yearID'] == year) & (mo['countyID'] == countyid)]

vmt_calc_06 = vmt_calc_06.pivot(index=['yearID','countyID','sourceTypeID'],columns='activitytypeID',values='activity')

vmt_calc_06 = vmt_calc_06.reset_index()

vmt_calc_06 = vmt_calc_06.rename(columns={1:'vmt', 6:'population'})

vmt_calc_06['ratio'] = vmt_calc_06['population'] / vmt_calc_06['vmt']

display(vmt_calc_06)

activitytypeID,yearID,countyID,sourceTypeID,vmt,population,ratio
0,2050,49049,11,35035,17174,0.490196
1,2050,49049,21,3270974,112182,0.034296
2,2050,49049,31,9787872,334535,0.034179
3,2050,49049,32,1002166,34175,0.034101
4,2050,49049,41,46086,816,0.017706
5,2050,49049,42,14355,262,0.018251
6,2050,49049,43,21005,1082,0.051512
7,2050,49049,51,7083,159,0.022448
8,2050,49049,52,653662,23285,0.035622
9,2050,49049,53,44078,1038,0.023549


## VMT Calc 7

In [19]:
vmt_calc_07 = filtered_sourcetype.loc[(filtered_sourcetype['sourceTypeID'] == 53) | (filtered_sourcetype['sourceTypeID'] == 62)]

display(vmt_calc_07)

,yearID,countyid,sourceTypeID,pop
15123,2050,49049,53,4748.74
15264,2050,49049,62,1875.23


## VMT Calc 8

In [20]:
bb = vmt_calc_05.loc[(vmt_calc_05['sourceTypeID'] == 53) | (vmt_calc_05['sourceTypeID'] == 62)]
bb = bb.loc[:,['yearID','sourceTypeID','vmt']]
dd = vmt_calc_06.loc[(vmt_calc_05['sourceTypeID'] == 53) | (vmt_calc_05['sourceTypeID'] == 62)]
dd = dd.loc[:,['yearID','sourceTypeID','ratio']]
cc = vmt_calc_07.loc[:,['yearID','sourceTypeID','pop']]

display(bb)
display(dd)
display(cc)

vmt_calc_08 = pd.merge(bb, dd[['sourceTypeID', 'ratio']], on='sourceTypeID', how='left')
vmt_calc_08 = pd.merge(vmt_calc_08, cc[['sourceTypeID', 'pop']], on='sourceTypeID', how='left')

vmt_calc_08.rename(columns={'pop':'oldpop'},inplace=True)
vmt_calc_08['newpop'] = (vmt_calc_08['ratio'] * vmt_calc_08['vmt'])

vmt_calc_08['updatedpop'] = np.maximum(vmt_calc_08['oldpop'],vmt_calc_08['newpop']).round()

#.round().astype(int)
display(vmt_calc_08)


,yearID,sourceTypeID,vmt
9,2050,53,110284.238857
12,2050,62,339691.830200


activitytypeID,yearID,sourceTypeID,ratio
9,2050,53,0.023549
12,2050,62,0.004689


,yearID,sourceTypeID,pop
15123,2050,53,4748.74
15264,2050,62,1875.23


,yearID,sourceTypeID,vmt,ratio,oldpop,newpop,updatedpop
0,2050,53,110284.238857,0.023549,4748.74,2597.101500,4749.0
1,2050,62,339691.830200,0.004689,1875.23,1592.699146,1875.0


## Load in New Pop from Pop Checker

In [21]:
display(filtered_sourcetype)
#sourcetypeyear_df = sourcetypeyear_df.replace(sourcetypeyear_df['pop'],vmt_calc_08['updatedpop'])
#filtered_sourcetype = filtered_sourcetype.replace(filtered_sourcetype['pop'],'test')
display(vmt_calc_08.loc[vmt_calc_08['sourceTypeID']== 53,'updatedpop'])
filtered_sourcetype.loc[filtered_sourcetype['sourceTypeID'] == 53,'pop'] = vmt_calc_08.loc[vmt_calc_08['sourceTypeID']== 53,'updatedpop'].values
filtered_sourcetype.loc[filtered_sourcetype['sourceTypeID'] == 62,'pop'] = vmt_calc_08.loc[vmt_calc_08['sourceTypeID']== 62,'updatedpop'].values
display(filtered_sourcetype)

,yearID,countyid,sourceTypeID,pop
14700,2050,49049,11,17227.400
14747,2050,49049,21,103123.000
14794,2050,49049,31,388007.000
14841,2050,49049,32,44533.100
14888,2050,49049,41,750.023
14935,2050,49049,42,105.885
14982,2050,49049,43,927.970
15029,2050,49049,51,167.327
15076,2050,49049,52,29472.900
15123,2050,49049,53,4748.740


0    4749.0
Name: updatedpop, dtype: float64

,yearID,countyid,sourceTypeID,pop
14700,2050,49049,11,17227.400
14747,2050,49049,21,103123.000
14794,2050,49049,31,388007.000
14841,2050,49049,32,44533.100
14888,2050,49049,41,750.023
14935,2050,49049,42,105.885
14982,2050,49049,43,927.970
15029,2050,49049,51,167.327
15076,2050,49049,52,29472.900
15123,2050,49049,53,4749.000


# Create Databases

In [22]:
#####Creates an input database. Use DAQ Load scripts to enter DAQ data.#####

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="moves",
        password="moves",
        host="127.0.0.1",
        port=3307,

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

#mycursor = cnx.cursor()

#mycursor.execute("CREATE DATABASE test_mysql_db")

#mycursor.execute("SHOW DATABASES")
#for db in mycursor:
  #print(db)

#conn = sqlite3.connect('test_db.db')
#cursor = conn.cursor()

tables = [
    """
    CREATE TABLE IF NOT EXISTS auditlog (
        whenHappened TEXT NOT NULL,
        importerName TEXT NOT NULL,
        briefDescription TEXT,
        fullDescription TEXT,
        PRIMARY KEY (whenHappened, importerName)
    );
    """,
    """

    CREATE TABLE IF NOT EXISTS avft (
        sourceTypeID INTEGER NOT NULL,
        modelYearID INTEGER NOT NULL,
        fuelTypeID INTEGER NOT NULL,
        engTechID INTEGER NOT NULL,
        fuelEngFraction REAL NOT NULL,
        PRIMARY KEY (sourceTypeID, modelYearID, fuelTypeID, engTechID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS avgspeeddistribution (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        roadTypeID INTEGER NOT NULL DEFAULT 0,
        hourDayID INTEGER NOT NULL DEFAULT 0,
        avgSpeedBinID INTEGER NOT NULL DEFAULT 0,
        avgSpeedFraction REAL,
        PRIMARY KEY (avgSpeedBinID, hourDayID, roadTypeID, sourceTypeID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS county (
        countyID INTEGER NOT NULL DEFAULT 0,
        stateID INTEGER NOT NULL DEFAULT 0,
        countyName TEXT,
        altitude TEXT,
        GPAFract REAL,
        barometricPressure REAL,
        barometricPressureCV REAL,
        countyTypeID INTEGER,
        msa TEXT,
        PRIMARY KEY (countyID, stateID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS dayvmtfraction (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        monthID INTEGER NOT NULL DEFAULT 0,
        roadTypeID INTEGER NOT NULL DEFAULT 0,
        dayID INTEGER NOT NULL DEFAULT 0,
        dayVMTFraction REAL,
        PRIMARY KEY (dayID, monthID, roadTypeID, sourceTypeID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS fuelformulation (
        fuelFormulationID INTEGER NOT NULL,
        fuelSubtypeID INTEGER NOT NULL DEFAULT 0,
        RVP REAL,
        sulfurLevel REAL NOT NULL DEFAULT 30,
        ETOHVolume REAL,
        MTBEVolume REAL,
        ETBEVolume REAL,
        TAMEVolume REAL,
        aromaticContent REAL,
        olefinContent REAL,
        benzeneContent REAL,
        e200 REAL,
        e300 REAL,
        volToWtPercentOxy REAL,
        BioDieselEsterVolume REAL,
        CetaneIndex REAL,
        PAHContent REAL,
        T50 REAL,
        T90 REAL,
        PRIMARY KEY (fuelFormulationID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS fuelsupply (
        fuelRegionID INTEGER NOT NULL DEFAULT 0,
        fuelYearID INTEGER NOT NULL DEFAULT 0,
        monthGroupID INTEGER NOT NULL DEFAULT 0,
        fuelFormulationID INTEGER NOT NULL,
        marketShare REAL,
        marketShareCV REAL,
        PRIMARY KEY (fuelRegionID, fuelFormulationID, monthGroupID, fuelYearID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS fuelusagefraction (
        countyID INTEGER NOT NULL,
        fuelYearID INTEGER NOT NULL,
        modelYearGroupID INTEGER NOT NULL,
        sourceBinFuelTypeID INTEGER NOT NULL,
        fuelSupplyFuelTypeID INTEGER NOT NULL,
        usageFraction REAL,
        PRIMARY KEY (countyID, fuelYearID, modelYearGroupID, sourceBinFuelTypeID, fuelSupplyFuelTypeID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS hotellingactivitydistribution (
        zoneID INTEGER NOT NULL,
        fuelTypeID INTEGER NOT NULL,
        beginModelYearID INTEGER NOT NULL,
        endModelYearID INTEGER NOT NULL,
        opModeID INTEGER NOT NULL,
        opModeFraction REAL NOT NULL,
        PRIMARY KEY (zoneID, fuelTypeID, beginModelYearID, endModelYearID, opModeID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS hotellingagefraction (
        zoneID INTEGER NOT NULL,
        ageID INTEGER NOT NULL,
        ageFraction REAL NOT NULL,
        PRIMARY KEY (zoneID, ageID)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS hotellinghourfraction (
        zoneID INTEGER NOT NULL,
        dayID INTEGER NOT NULL,
        hourID INTEGER NOT NULL,
        hourFraction REAL NOT NULL,
        PRIMARY KEY (zoneID, dayID, hourID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS hotellinghoursperday (
        yearID INTEGER NOT NULL,
        zoneID INTEGER NOT NULL,
        dayID INTEGER NOT NULL,
        hotellingHoursPerDay REAL NOT NULL,
        PRIMARY KEY (yearID, zoneID, dayID)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS hotellingmonthadjust (
        zoneID INTEGER NOT NULL,
        monthID INTEGER NOT NULL,
        monthAdjustment REAL NOT NULL,
        PRIMARY KEY (zoneID, monthID)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS hourvmtfraction (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        roadTypeID INTEGER NOT NULL DEFAULT 0,
        dayID INTEGER NOT NULL DEFAULT 0,
        hourID INTEGER NOT NULL DEFAULT 0,
        hourVMTFraction REAL,
        PRIMARY KEY (dayID, hourID, roadTypeID, sourceTypeID)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS hpmsvtypeday (
        yearID INTEGER NOT NULL,
        monthID INTEGER NOT NULL,
        dayID INTEGER NOT NULL,
        HPMSVtypeID INTEGER NOT NULL,
        VMT REAL NOT NULL,
        PRIMARY KEY (yearID, monthID, dayID, HPMSVtypeID)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS hpmsvtypeyear (
        HPMSVtypeID INTEGER NOT NULL DEFAULT 0,
        yearID INTEGER NOT NULL DEFAULT 0,
        VMTGrowthFactor REAL,
        HPMSBaseYearVMT REAL,
        PRIMARY KEY (HPMSVtypeID, yearID)
    );
    """,
    
    """
    CREATE TABLE IF NOT EXISTS idledayadjust (
        sourceTypeID INTEGER NOT NULL,
        dayID INTEGER NOT NULL,
        idleDayAdjust REAL NOT NULL,
        PRIMARY KEY (sourceTypeID, dayID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS idlemodelyeargrouping (
        sourceTypeID INTEGER NOT NULL,
        minModelYearID INTEGER NOT NULL,
        maxModelYearID INTEGER NOT NULL,
        totalIdleFraction REAL NOT NULL,
        PRIMARY KEY (sourceTypeID, minModelYearID, maxModelYearID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS idlemonthadjust (
        sourceTypeID INTEGER NOT NULL,
        monthID INTEGER NOT NULL,
        idleMonthAdjust REAL NOT NULL,
        PRIMARY KEY (sourceTypeID, monthID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS imcoverage (
        polProcessID INTEGER NOT NULL DEFAULT 0,
        stateID INTEGER NOT NULL DEFAULT 0,
        countyID INTEGER NOT NULL DEFAULT 0,
        yearID INTEGER NOT NULL DEFAULT 0,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        fuelTypeID INTEGER NOT NULL DEFAULT 0,
        IMProgramID INTEGER NOT NULL DEFAULT 0,
        begModelYearID INTEGER NOT NULL DEFAULT 0,
        endModelYearID INTEGER NOT NULL DEFAULT 0,
        inspectFreq INTEGER,
        testStandardsID INTEGER NOT NULL DEFAULT 0,
        useIMyn TEXT NOT NULL DEFAULT 'Y',
        complianceFactor REAL,
        PRIMARY KEY (polProcessID, stateID, countyID, yearID, sourceTypeID, fuelTypeID, IMProgramID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS monthvmtfraction (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        monthID INTEGER NOT NULL DEFAULT 0,
        monthVMTFraction REAL,
        PRIMARY KEY (monthID, sourceTypeID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS onroadretrofit (
        pollutantID INTEGER NOT NULL,
        processID INTEGER NOT NULL,
        fuelTypeID INTEGER NOT NULL,
        sourceTypeID INTEGER NOT NULL,
        retrofitYearID INTEGER NOT NULL,
        beginModelYearID INTEGER NOT NULL,
        endModelYearID INTEGER NOT NULL,
        cumFractionRetrofit REAL NOT NULL DEFAULT 0,
        retrofitEffectiveFraction REAL NOT NULL DEFAULT 0,
        PRIMARY KEY (pollutantID, processID, fuelTypeID, sourceTypeID, retrofitYearID, beginModelYearID, endModelYearID)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS roadtypedistribution (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        roadTypeID INTEGER NOT NULL DEFAULT 0,
        roadTypeVMTFraction REAL,
        PRIMARY KEY (roadTypeID, sourceTypeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS sourcetypeagedistribution (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        yearID INTEGER NOT NULL DEFAULT 0,
        ageID INTEGER NOT NULL DEFAULT 0,
        ageFraction REAL,
        PRIMARY KEY (ageID, sourceTypeID, yearID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS sourcetypedayvmt (
        yearID INTEGER NOT NULL,
        monthID INTEGER NOT NULL,
        dayID INTEGER NOT NULL,
        sourceTypeID INTEGER NOT NULL,
        VMT REAL NOT NULL,
        PRIMARY KEY (yearID, monthID, dayID, sourceTypeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS sourcetypeyear (
        yearID INTEGER NOT NULL DEFAULT 0,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        salesGrowthFactor REAL,
        sourceTypePopulation REAL,
        migrationrate REAL,
        PRIMARY KEY (sourceTypeID, yearID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS sourcetypeyearvmt (
        yearID INTEGER NOT NULL,
        sourceTypeID INTEGER NOT NULL,
        VMT REAL NOT NULL,
        PRIMARY KEY (yearID, sourceTypeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS starts (
        hourDayID INTEGER NOT NULL DEFAULT 0,
        monthID INTEGER NOT NULL DEFAULT 0,
        yearID INTEGER NOT NULL DEFAULT 0,
        ageID INTEGER NOT NULL DEFAULT 0,
        zoneID INTEGER NOT NULL DEFAULT 0,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        starts REAL,
        StartsCV REAL,
        isUserInput TEXT NOT NULL DEFAULT 'N',
        PRIMARY KEY (ageID, hourDayID, monthID, sourceTypeID, yearID, zoneID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS startsageadjustment (
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        ageID INTEGER NOT NULL DEFAULT 0,
        ageAdjustment REAL,
        PRIMARY KEY (sourceTypeID, ageID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS startshourfraction (
        dayID INTEGER NOT NULL,
        hourID INTEGER NOT NULL,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        allocationFraction REAL NOT NULL,
        PRIMARY KEY (dayID, hourID, sourceTypeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS startsmonthadjust (
        monthID INTEGER NOT NULL,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        monthAdjustment REAL NOT NULL,
        PRIMARY KEY (monthID, sourceTypeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS startsopmodedistribution (
        dayID INTEGER NOT NULL DEFAULT 0,
        hourID INTEGER NOT NULL DEFAULT 0,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        ageID INTEGER NOT NULL DEFAULT 0,
        opModeID INTEGER NOT NULL DEFAULT 0,
        opModeFraction REAL,
        isUserInput TEXT NOT NULL DEFAULT 'N',
        PRIMARY KEY (dayID, hourID, sourceTypeID, ageID, opModeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS startsperday (
        dayID INTEGER NOT NULL DEFAULT 0,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        startsPerDay REAL,
        PRIMARY KEY (sourceTypeID, dayID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS startsperdaypervehicle (
        dayID INTEGER NOT NULL DEFAULT 0,
        sourceTypeID INTEGER NOT NULL DEFAULT 0,
        startsPerDayPerVehicle REAL,
        PRIMARY KEY (sourceTypeID, dayID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS state (
        stateID INTEGER NOT NULL DEFAULT 0,
        stateName TEXT DEFAULT NULL,
        stateAbbr TEXT DEFAULT NULL,
        idleRegionID INTEGER DEFAULT NULL,
        PRIMARY KEY (stateID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS totalidlefraction (
        sourceTypeID INTEGER NOT NULL,
        minModelYearID INTEGER NOT NULL,
        maxModelYearID INTEGER NOT NULL,
        monthID INTEGER NOT NULL,
        dayID INTEGER NOT NULL,
        idleRegionID INTEGER NOT NULL,
        countyTypeID INTEGER NOT NULL,
        totalIdleFraction REAL,
        PRIMARY KEY (sourceTypeID, minModelYearID, maxModelYearID, monthID, dayID, idleRegionID, countyTypeID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS year (
        yearID INTEGER NOT NULL DEFAULT 0,
        isBaseYear TEXT DEFAULT NULL,
        fuelYearID INTEGER NOT NULL DEFAULT 0,
        PRIMARY KEY (yearID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS zone (
        zoneID INTEGER NOT NULL DEFAULT 0,
        countyID INTEGER NOT NULL DEFAULT 0,
        startAllocFactor REAL,
        idleAllocFactor REAL,
        SHPAllocFactor REAL,
        PRIMARY KEY (zoneID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS zonemonthhour (
        monthID INTEGER NOT NULL DEFAULT 0,
        zoneID INTEGER NOT NULL DEFAULT 0,
        hourID INTEGER NOT NULL DEFAULT 0,
        temperature REAL,
        relHumidity REAL,
        heatIndex REAL,
        specificHumidity REAL,
        molWaterFraction REAL,
        PRIMARY KEY (hourID, monthID, zoneID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS zoneroadtype (
        zoneID INTEGER NOT NULL DEFAULT 0,
        roadTypeID INTEGER NOT NULL DEFAULT 0,
        SHOAllocFactor REAL,
        PRIMARY KEY (roadTypeID, zoneID)
    );
    """
]
# Execute table creation statements
#for table in tables:
    #cursor.execute(table)

# Commit changes and close connection
#conn.commit()
#conn.close()

# Establish a connection to the MySQL server
conn = mysql.connector.connect(
    host="localhost",  # Replace with your MySQL server host
    user="root",  # Replace with your MySQL username
    password="Starfish-7Favor"  # Replace with your MySQL password
)

# Create a cursor object
cursor = conn.cursor()

# Create a new database
cursor.execute("CREATE DATABASE mydatabase_pythontest")  # Replace 'mydatabase' with your desired database name

# Confirm database creation
cursor.execute("SHOW DATABASES")
for db in cursor:
    print(db)

# Close the cursor and connection
cursor.close()
conn.close()

SystemError: <method '_read_response' of 'mariadb.connection' objects> returned a result with an exception set